In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
import keras.backend as K
from keras import layers, models, optimizers
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import Huber

import gc
import os
import pickle
import warnings

# os.environ['KMP_DUPLICATE_LIB_OK']='True'
warnings.filterwarnings('ignore')
tf.config.experimental.set_visible_devices([], 'GPU')


In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss, accuracy_score, f1_score
from sklearn.decomposition import PCA

from pystacknet.pystacknet import StackNetClassifier



In [10]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from bayes_opt import BayesianOptimization


In [11]:
tr = pd.read_csv('./data/train.csv')
te = pd.read_csv('./data/test.csv')

sub = pd.read_csv('./data/sample_submission.csv', index_col=0)

In [12]:
# target_lbe = LabelEncoder().fit(target)

# t = target_lbe.transform(target)

column_number = {}
for i, column in enumerate(sub.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

tr['type_num'] = tr['type'].apply(lambda x: to_number(x, column_number))



In [13]:
target = tr['type_num']
t = target.copy()

train_X = tr.drop(['id', 'type', 'type_num', 'fiberID'], axis=1)
test_X = te.drop(['id','fiberID'], axis=1)

# del train_X, test_X, tr, te
train_X = (train_X - np.mean(train_X))/np.std(train_X)
test_X = (test_X - np.mean(test_X))/np.std(test_X)



In [7]:
f = open('./res_X.bin', 'rb')
res_X = pickle.load(f)
f.close()

f = open('./rte_X.bin', 'rb')
rte_X = pickle.load(f)
f.close()


In [31]:
K.clear_session()

inputs = layers.Input(shape=(train_X.shape[1], ))

x = layers.Dense(1024, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.advanced_activations.LeakyReLU(0.3)(x)

x = layers.Dense(1024, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(x)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.advanced_activations.LeakyReLU(0.3)(x)

x = layers.Dense(512, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(x)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.advanced_activations.LeakyReLU(0.3)(x)

x = layers.Dense(256, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(x)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.advanced_activations.LeakyReLU(0.3)(x)

x = layers.Dense(128, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(x)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.advanced_activations.LeakyReLU(0.3)(x)

x = layers.Dense(64, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(x)
x = layers.BatchNormalization(momentum=0.8)(x)
x = layers.advanced_activations.LeakyReLU(0.3)(x)

x = layers.Dense(19, activation='softmax')(x)

m = models.Model(inputs, x)

# sparse_categorical_crossentropy
m.compile(optimizer = 'adam',
         loss = 'sparse_categorical_crossentropy', 
          metrics = ['acc'])

In [32]:
es = EarlyStopping(patience=20, restore_best_weights=True)
def schedule(epoch):
    if epoch < 50:
        return 0.001
    elif epoch < 100:
        return 0.0005
    else:
        return 0.00001
lrs = LearningRateScheduler(schedule)

In [33]:
m.fit(train_X, t, 
     epochs = 300,
     validation_split=0.1,
     batch_size=1024*16,
      callbacks=[es, lrs]
     )

Train on 179991 samples, validate on 20000 samples
Epoch 1/300
179991/179991 [==============================] - 20s 109us/step - loss: 3.2189 - acc: 0.3358 - val_loss: 3.4468 - val_acc: 0.2046
Epoch 2/300
179991/179991 [==============================] - 19s 105us/step - loss: 2.7948 - acc: 0.5080 - val_loss: 3.2738 - val_acc: 0.2932
Epoch 3/300
179991/179991 [==============================] - 19s 105us/step - loss: 2.5338 - acc: 0.5323 - val_loss: 3.1650 - val_acc: 0.1931
Epoch 4/300
179991/179991 [==============================] - 19s 104us/step - loss: 2.2532 - acc: 0.5458 - val_loss: 2.7703 - val_acc: 0.4567
Epoch 5/300
179991/179991 [==============================] - 19s 104us/step - loss: 2.0874 - acc: 0.5761 - val_loss: 2.7078 - val_acc: 0.4168
Epoch 6/300
179991/179991 [==============================] - 19s 104us/step - loss: 2.0080 - acc: 0.6229 - val_loss: 2.8412 - val_acc: 0.2768
Epoch 7/300
179991/179991 [==============================] - 19s 104us/step - loss: 1.8637 - acc:

KeyboardInterrupt: 

In [28]:
y_pred = m.predict(test_X)
submission = pd.DataFrame(data=y_pred, columns=sub.columns, index=sub.index)
submission.to_csv('./sub/nn8.csv', index=True)


In [55]:
m.evaluate(res_X, t)

199991/199991 [==============================] - 4s 19us/step


[0.2791271693419825, 0.9226264953613281]

In [34]:
m.evaluate(train_X, t)

199991/199991 [==============================] - 22s 110us/step


[2.944787236940841, 0.2723722457885742]

In [ ]:
# kk = pd.DataFrame(lgb_clf.predict_proba(te_X), columns=target_lbe.classes_)
# sub[sub.columns[1:]] = kk[sub.columns[1:]]

In [ ]:
# sub.to_csv('./sub/stk1.csv', index=False)

In [ ]:
lgb_imp = lgb_clf.feature_importances_

lgb_imp_idx = []
cols = tr_X.columns
for i, imp in enumerate(lgb_imp):
    if imp > 0:
        lgb_imp_idx.append(i)
        
new_cols_imp = cols[lgb_imp_idx]
new_cols_imp = list(new_cols_imp)

In [ ]:
print(len(tr_X.columns), len(new_cols_imp))

In [ ]:
f = open('./new_cols.bin', 'wb')
pickle.dump(new_cols_imp, f)
f.close()

In [19]:
r = pd.read_csv('./sub/sk_fin2.csv')

In [27]:
rte_X = r[r.columns[1:]].values

In [25]:
rte_X[0]

array([1.33017038e-06, 2.81874312e-06, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.48302233e-07, 5.48496696e-05, 2.07572390e-04,
       9.87961760e-01, 1.54013286e-05, 8.96521114e-06, 2.78091580e-04,
       0.00000000e+00, 4.65492545e-04, 2.59516746e-06, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.10008749e-02])